# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [13]:
#Your code here
import pandas as pd
data = pd.read_csv("homepage_actions.csv")
viewers = data[data["action"] == "view"]
clickers = data[data["action"] == "click"]
# Number of viewers who also clicked
num_clickers = clickers['id'].nunique()
print("Number of viewers who also clicked:", num_clickers)

# Check for anomalies (anyone clicked without viewing)
anomalies = clickers[~clickers['id'].isin(viewers['id'])]
print("Anomalies - Clicked without viewing:")
print(anomalies)

# Check for overlap between control and experiment groups
control_group = data[data['group'] == 'control']
experiment_group = data[data['group'] == 'experiment']
overlap = control_group[control_group['id'].isin(experiment_group['id'])]
print("Overlap between control and experiment groups:")
print(overlap)

# Account for overlap in experimental design
# Remove overlapping data points or analyze them separately

Number of viewers who also clicked: 1860
Anomalies - Clicked without viewing:
Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []
Overlap between control and experiment groups:
Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [14]:
from scipy.stats import ttest_ind

control_clicks = control_group['action'].value_counts()['click']
experiment_clicks = experiment_group['action'].value_counts()['click']

# Perform a t-test for independent samples
t_stat, p_value = ttest_ind(control_group['action'] == 'click', experiment_group['action'] == 'click')

# Interpret the results
if p_value < 0.05:
    print("The experimental homepage was more effective than the control group.")
else:
    print("There is no significant difference between the experimental and control groups.")


There is no significant difference between the experimental and control groups.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [15]:
#Your code here
control_rate = control_clicks / len(control_group)
expected_clicks = control_rate * len(experiment_group)

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [16]:
#Your code here
import numpy as np
variance = len(experiment_group) * control_rate * (1 - control_rate)
std_dev = np.sqrt(variance)
actual_clicks = experiment_clicks
actual_clicks

928

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [17]:
#Your code here
from scipy.stats import norm
z_score = (actual_clicks - expected_clicks) / std_dev
p_value = 1 - norm.cdf(z_score)

# Interpret the results
if p_value < 0.05:
    print("The p-value indicates that the experimental homepage was significantly more effective.")
else:
    print("The p-value indicates that there is no significant difference between the experimental and control groups.")

The p-value indicates that the experimental homepage was significantly more effective.


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **No, there was a significant difference between the two tests**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.